In [1]:
!pip -q install txtai[all] langchain pypdf > /dev/null

In [2]:
%%capture

from langchain.text_splitter import RecursiveCharacterTextSplitter
from txtai.embeddings import Embeddings
from langchain.document_loaders import PyPDFLoader
import glob

In [ ]:
!git clone https://github.com/twitter/the-algorithm # replace any repository of your choice

In [4]:
import os
from langchain.document_loaders import TextLoader

root_dir = './the-algorithm'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [5]:
document_splitter = RecursiveCharacterTextSplitter(chunk_size=350,
                                                   chunk_overlap=25,
                                                   length_function=len)

In [8]:
split_data = []
for doc in docs:
  #print(docs)
  temp_split = document_splitter.split_text(doc.page_content)
  split_data.extend(temp_split)

In [9]:
len(split_data)

79720

In [13]:
# Create embeddings index with content enabled. The default behavior is to only store indexed vectors.
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2", 
                         "content": True, 
                         "objects": True})

In [14]:
# Create an index for the list of text
embeddings.index([(uid, 
                   text, 
                   None) for uid, text in enumerate(split_data[:500])])

In [15]:
embeddings.save("algo_index")

In [23]:
data_retrieved = embeddings.search("What are the top classes?",4)

In [17]:
# Submits a series of prompts to the Hugging Face API.
# This call can easily be switched to use the OpenAI API (GPT-3), Cohere API or a library like langchain.
import requests

def api(prompts):
  response = requests.post("https://api-inference.huggingface.co/models/google/flan-t5-base",
                           json={"inputs": prompts})

  return [x["generated_text"] for x in response.json()]

In [18]:
from txtai.pipeline import Extractor

# Create extractor instance, submit prompts to the Hugging Face inference API
extractor = Extractor(embeddings, api)

In [19]:
def prompt(question):
  return f"""
    Answer the following question using the context below.
    Question: {question}
    Context:
  """

In [30]:
question = "What is clusterTweet"

In [ ]:
from langchain.vectorstores import FAISS

